In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import pickle

C:\Users\hskay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_PATH = "meta-llama/Llama-2-7b-chat-hf"
print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    dtype=torch.float16,
    device_map='auto' if torch.cuda.is_available() else None
)
model.eval()


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.65s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [6]:
model.parameters

<bound method Module.parameters of LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNor

In [4]:
# Helper function to capture activations
def get_activation(name, activations_dict):
    def hook(module, input, output):
        # Handle different output types
        if isinstance(output, tuple):
            activations_dict[name] = output[0].detach().clone()
        else:
            activations_dict[name] = output.detach().clone()
    return hook

In [ ]:
# Generate sample inputs for analysis
def generate_sample_inputs(tokenizer, seq_length=32):    
    sample_texts = [
        "The quick brown fox jumps over the lazy dog.",
        "Artificial intelligence is transforming the world of technology.",
        "In a hole in the ground there lived a hobbit.",
        "To be or not to be, that is the question Shakespeare posed.",
        "Machine learning models require large datasets for training.",
        "The mitochondria is the powerhouse of the cell in biology.",
        "Climate change is causing unprecedented shifts in global weather patterns.",
        "Mozart composed his first symphony at the age of eight years old.",
        "The stock market experienced significant volatility during the pandemic crisis.",
        "Quantum physics reveals the strange behavior of particles at subatomic levels.",
        "Professional chefs recommend using fresh herbs to enhance flavor profiles.",
        "Ancient Egyptian pyramids were built using sophisticated engineering techniques.",
        "Regular exercise and proper nutrition are essential for maintaining good health.",
        "The International Space Station orbits Earth approximately every ninety minutes.",
        "Cryptocurrency markets operate twenty-four hours a day across global exchanges.",
        "Vincent van Gogh painted Starry Night while staying at an asylum.",
        "Professional athletes must maintain strict training regimens throughout their careers.",
        "The Amazon rainforest produces twenty percent of the world's oxygen supply.",
        "Modern architecture emphasizes clean lines and functional design principles.",
        "Forensic scientists use DNA analysis to solve complex criminal investigations.",
        "Traditional Japanese tea ceremonies follow centuries-old ritualistic practices.",
        "Marine biologists study coral reef ecosystems threatened by ocean acidification.",
        "The Renaissance period marked a cultural rebirth in European art and science.",
        "Cybersecurity experts work tirelessly to protect digital infrastructure from threats.",
        "Sustainable agriculture practices help preserve soil quality for future generations."
    ]
    
    inputs = []
    for i in range(len(sample_texts)):
        # Cycle through sample texts and add variations
        base_text = sample_texts[i % len(sample_texts)]

        # Tokenize
        tokenized = tokenizer(
            base_text, 
            return_tensors="pt", 
            padding='max_length', 
            truncation=True, 
            max_length=seq_length
        )
        inputs.append(tokenized.input_ids.to(model.device))
    
    return inputs

In [ ]:
# Generate activation differences for Llama-2 layers
def generate_activation_differences_llama(model, X_data, n_samples=50, n_reconstructions=3):
    results = []
    
    # Select specific layers to analyze (first few transformer layers)
    # layer_names = [
    #     'model.layers.0',  # First transformer layer
    #     'model.layers.1',  # Second transformer layer  
    #     'model.layers.2',  # Third transformer layer
    # ]
    layer_names = [f'model.layers.{i}' for i in range(len(model.model.layers))]
    for sample_idx in tqdm(range(min(n_samples, len(X_data))), desc="Processing samples"):
        original_input = X_data[sample_idx]
        
        # Get original activations
        original_activations = {}
        hooks = []
        
       # Register hooks for all layers
        for layer_name in layer_names:
            layer_module = model
            for attr in layer_name.split('.'):
                layer_module = getattr(layer_module, attr)
            hooks.append(layer_module.register_forward_hook(
                get_activation(layer_name, original_activations)
            ))
        
        # Get original output and activations
        with torch.no_grad():
            original_output = model(original_input).logits
        
        # Remove hooks
        for hook in hooks:
            hook.remove()
        
        # Multiple reconstruction attempts
        for recon_idx in range(n_reconstructions):
            # Initialize random input embeddings for reconstruction
            seq_length = original_input.shape[1]
            embedding_dim = model.config.hidden_size
            
            # Use embeddings instead of token IDs for gradient-based optimization
            reconstructed_embeddings = torch.randn(
                1, seq_length, embedding_dim,
                device=model.device,
                dtype=torch.float32,
                requires_grad=True
            )
            
            optimizer = optim.Adam([reconstructed_embeddings], lr=0.001)
            
            # Reconstruction optimization
            for iteration in range(10000): 
                optimizer.zero_grad()
                
                # Forward pass with embeddings
                embeddings_model_dtype = reconstructed_embeddings.to(model.dtype)
                output = model(inputs_embeds=embeddings_model_dtype).logits
                
                # Loss: match original output
                loss = nn.functional.mse_loss(output.float(), original_output.float())
                
                # Add regularization
                reg_loss = 0.001 * torch.mean(reconstructed_embeddings ** 2)
                total_loss = loss + reg_loss
                
                total_loss.backward()
                optimizer.step()
                
                if total_loss.item() < 1e-3:
                    break
                if iteration % 1000 == 0:
                    print(f"Sample {sample_idx}, Recon {recon_idx}, Iter {iteration}, Loss: {total_loss.item():.6f}",end="")
            
            # Get reconstructed activations
            reconstructed_activations = {}
            hooks = []
            
            for layer_name in layer_names:
                layer_module = model
                for attr in layer_name.split('.'):
                    layer_module = getattr(layer_module, attr)
                hooks.append(layer_module.register_forward_hook(
                    get_activation(layer_name, reconstructed_activations)
                ))
            
            with torch.no_grad():
                embeddings_model_dtype = reconstructed_embeddings.to(model.dtype)
                _ = model(inputs_embeds=embeddings_model_dtype)
            
            # Remove hooks
            for hook in hooks:
                hook.remove()
            
            # Calculate differences for each layer
            row = {'sample_idx': sample_idx, 'reconstruction_idx': recon_idx}
            
            # Store individual layer metrics
            all_layer_max_diffs = []
            
            for layer_name in layer_names:
                if layer_name in original_activations and layer_name in reconstructed_activations:
                    orig_act = original_activations[layer_name].flatten().float()
                    recon_act = reconstructed_activations[layer_name].flatten().float()
                    
                    abs_diff = torch.abs(orig_act - recon_act)
                    
                    layer_short = layer_name.split('.')[-1]  # Get layer number
                    row[f'layer_{layer_short}_min_abs_diff'] = abs_diff.min().item()
                    row[f'layer_{layer_short}_mean_abs_diff'] = abs_diff.mean().item()
                    row[f'layer_{layer_short}_max_abs_diff'] = abs_diff.max().item()
                    
                    all_layer_max_diffs.append(abs_diff.max().item())
            
            # Store the maximum difference across ALL layers
            if all_layer_max_diffs:
                row['all_layers_max_diff'] = max(all_layer_max_diffs)
                row['all_layers_min_of_max'] = min(all_layer_max_diffs)
            
            results.append(row)
    
    return pd.DataFrame(results)

In [ ]:
# Generate sample data
print("Generating sample inputs...")
X_data = generate_sample_inputs(tokenizer, seq_length=15) 
print(f"Generated {len(X_data)} samples")

Generating sample inputs...
Generated 20 samples


In [ ]:
# Generate results
print("Generating activation differences for Llama-2 layers...")
results = generate_activation_differences_llama(model, X_data, n_samples=25, n_reconstructions=25)
#406m 40.9s

Generating activation differences for Llama-2 layers...


Processing samples:   0%|          | 0/10 [00:00<?, ?it/s]

Sample 0, Recon 0, Iter 0, Loss: 18.072140
Sample 0, Recon 0, Iter 100, Loss: 6.001960
Sample 0, Recon 0, Iter 200, Loss: 4.446840
Sample 0, Recon 0, Iter 300, Loss: 3.604485
Sample 0, Recon 0, Iter 400, Loss: 3.302956
Sample 0, Recon 0, Iter 500, Loss: 3.122141
Sample 0, Recon 0, Iter 600, Loss: 2.971247
Sample 0, Recon 0, Iter 700, Loss: 2.826409
Sample 0, Recon 0, Iter 800, Loss: 2.680085
Sample 0, Recon 0, Iter 900, Loss: 2.532717
Sample 0, Recon 0, Iter 1000, Loss: 2.393804
Sample 0, Recon 0, Iter 1100, Loss: 2.269571
Sample 0, Recon 0, Iter 1200, Loss: 2.159087
Sample 0, Recon 0, Iter 1300, Loss: 2.060229
Sample 0, Recon 0, Iter 1400, Loss: 1.970483
Sample 0, Recon 0, Iter 1500, Loss: 1.888022
Sample 0, Recon 0, Iter 1600, Loss: 1.811693
Sample 0, Recon 0, Iter 1700, Loss: 1.740411
Sample 0, Recon 0, Iter 1800, Loss: 1.674420
Sample 0, Recon 0, Iter 1900, Loss: 1.613055
Sample 0, Recon 0, Iter 2000, Loss: 1.555785
Sample 0, Recon 0, Iter 2100, Loss: 1.502473
Sample 0, Recon 0, It

Processing samples:  10%|█         | 1/10 [40:42<6:06:23, 2442.61s/it]

Sample 1, Recon 0, Iter 0, Loss: 18.817217
Sample 1, Recon 0, Iter 100, Loss: 6.056330
Sample 1, Recon 0, Iter 200, Loss: 4.589106
Sample 1, Recon 0, Iter 300, Loss: 3.701640
Sample 1, Recon 0, Iter 400, Loss: 3.357178
Sample 1, Recon 0, Iter 500, Loss: 3.164711
Sample 1, Recon 0, Iter 600, Loss: 3.012650
Sample 1, Recon 0, Iter 700, Loss: 2.877337
Sample 1, Recon 0, Iter 800, Loss: 2.747234
Sample 1, Recon 0, Iter 900, Loss: 2.616783
Sample 1, Recon 0, Iter 1000, Loss: 2.485771
Sample 1, Recon 0, Iter 1100, Loss: 2.362695
Sample 1, Recon 0, Iter 1200, Loss: 2.257669
Sample 1, Recon 0, Iter 1300, Loss: 2.167194
Sample 1, Recon 0, Iter 1400, Loss: 2.085522
Sample 1, Recon 0, Iter 1500, Loss: 2.009644
Sample 1, Recon 0, Iter 1600, Loss: 1.937713
Sample 1, Recon 0, Iter 1700, Loss: 1.869321
Sample 1, Recon 0, Iter 1800, Loss: 1.804370
Sample 1, Recon 0, Iter 1900, Loss: 1.742591
Sample 1, Recon 0, Iter 2000, Loss: 1.684048
Sample 1, Recon 0, Iter 2100, Loss: 1.629177
Sample 1, Recon 0, It

...

Processing samples:  80%|████████  | 8/10 [5:25:16<1:21:35, 2447.70s/it]

Sample 8, Recon 0, Iter 0, Loss: 15.227189
Sample 8, Recon 0, Iter 100, Loss: 4.409702
Sample 8, Recon 0, Iter 200, Loss: 3.831912
Sample 8, Recon 0, Iter 300, Loss: 3.493089
Sample 8, Recon 0, Iter 400, Loss: 3.194493
Sample 8, Recon 0, Iter 500, Loss: 3.004460
Sample 8, Recon 0, Iter 600, Loss: 2.870986
Sample 8, Recon 0, Iter 700, Loss: 2.754467
Sample 8, Recon 0, Iter 800, Loss: 2.637343
Sample 8, Recon 0, Iter 900, Loss: 2.508688
Sample 8, Recon 0, Iter 1000, Loss: 2.367514
Sample 8, Recon 0, Iter 1100, Loss: 2.236184
Sample 8, Recon 0, Iter 1200, Loss: 2.126427
Sample 8, Recon 0, Iter 1300, Loss: 2.035750
Sample 8, Recon 0, Iter 1400, Loss: 1.956011
Sample 8, Recon 0, Iter 1500, Loss: 1.883764
Sample 8, Recon 0, Iter 1600, Loss: 1.816899
Sample 8, Recon 0, Iter 1700, Loss: 1.754778
Sample 8, Recon 0, Iter 1800, Loss: 1.696709
Sample 8, Recon 0, Iter 1900, Loss: 1.642534
Sample 8, Recon 0, Iter 2000, Loss: 1.592100
Sample 8, Recon 0, Iter 2100, Loss: 1.544813
Sample 8, Recon 0, It

Processing samples:  90%|█████████ | 9/10 [6:06:07<40:48, 2448.88s/it]  

Sample 9, Recon 0, Iter 0, Loss: 20.640848
Sample 9, Recon 0, Iter 100, Loss: 5.178938
Sample 9, Recon 0, Iter 200, Loss: 4.053443
Sample 9, Recon 0, Iter 300, Loss: 3.619526
Sample 9, Recon 0, Iter 400, Loss: 3.291709
Sample 9, Recon 0, Iter 500, Loss: 3.033714
Sample 9, Recon 0, Iter 600, Loss: 2.857104
Sample 9, Recon 0, Iter 700, Loss: 2.701177
Sample 9, Recon 0, Iter 800, Loss: 2.563535
Sample 9, Recon 0, Iter 900, Loss: 2.440037
Sample 9, Recon 0, Iter 1000, Loss: 2.327774
Sample 9, Recon 0, Iter 1100, Loss: 2.226061
Sample 9, Recon 0, Iter 1200, Loss: 2.134440
Sample 9, Recon 0, Iter 1300, Loss: 2.051748
Sample 9, Recon 0, Iter 1400, Loss: 1.976199
Sample 9, Recon 0, Iter 1500, Loss: 1.906599
Sample 9, Recon 0, Iter 1600, Loss: 1.841429
Sample 9, Recon 0, Iter 1700, Loss: 1.779921
Sample 9, Recon 0, Iter 1800, Loss: 1.722325
Sample 9, Recon 0, Iter 1900, Loss: 1.668058
Sample 9, Recon 0, Iter 2000, Loss: 1.616875
Sample 9, Recon 0, Iter 2100, Loss: 1.568705
Sample 9, Recon 0, It

Processing samples: 100%|██████████| 10/10 [6:46:40<00:00, 2440.09s/it]


In [9]:
# Save results
results.to_csv('llama2_activation_diff_results.csv', index=False)
print(f"Results saved. Shape: {results.shape}")
print("\nFirst few rows:")
print(results.head())

Results saved. Shape: (30, 13)

First few rows:
   sample_idx  reconstruction_idx  layer_0_min_abs_diff  \
0           0                   0              0.000000   
1           0                   1              0.000008   
2           0                   2              0.000015   
3           1                   0              0.000019   
4           1                   1              0.000061   

   layer_0_mean_abs_diff  layer_0_max_abs_diff  layer_1_min_abs_diff  \
0               0.803223              6.522461              0.000000   
1               0.812638              5.780273              0.000000   
2               0.808089              6.396973              0.000061   
3               0.808671              6.916016              0.000015   
4               0.801016              4.633301              0.000000   

   layer_1_mean_abs_diff  layer_1_max_abs_diff  layer_2_min_abs_diff  \
0               0.830116            752.273438              0.000092   
1               0.83